In [4]:
library(lavaan)

In [96]:
raw_data <- read.csv('../data/WB_BGD_201805.csv')
data <- subset(raw_data, year >= 1986 & year <= 2015)
log_data <- log(data)

In [97]:
summary(data)
order = c('ISG', 'TO', 'TI', 'P_GDP', 'EI')
cor(data, use='pairwise')[order, order]

      year           ISG              TO            P_GDP       
 Min.   :1986   Min.   :20.05   Min.   :16.69   Min.   : 227.4  
 1st Qu.:1993   1st Qu.:22.86   1st Qu.:22.93   1st Qu.: 293.2  
 Median :2000   Median :23.92   Median :28.30   Median : 401.6  
 Mean   :2000   Mean   :24.07   Mean   :30.56   Mean   : 498.8  
 3rd Qu.:2008   3rd Qu.:25.90   3rd Qu.:39.48   3rd Qu.: 597.1  
 Max.   :2015   Max.   :28.15   Max.   :48.11   Max.   :1210.2  
                                                                
       EI               TI       
 Min.   :0.2049   Min.   : 93.0  
 1st Qu.:0.3205   1st Qu.:138.0  
 Median :0.3463   Median :291.0  
 Mean   :0.3513   Mean   :239.2  
 3rd Qu.:0.4179   3rd Qu.:317.5  
 Max.   :0.4893   Max.   :354.0  
 NA's   :1                       

,ISG,TO,TI,P_GDP,EI
ISG,1.0000000,0.9327355,0.7972720,0.8715645,-0.8552086
TO,0.9327355,1.0000000,0.8417575,0.8757430,-0.9187085
TI,0.7972720,0.8417575,1.0000000,0.6928933,-0.7820815
P_GDP,0.8715645,0.8757430,0.6928933,1.0000000,-0.9492223
EI,-0.8552086,-0.9187085,-0.7820815,-0.9492223,1.0000000


In [126]:
model1 <- '
# path model
EI ~ p_ei_isg*ISG + p_ei_to*TO + p_ei_ti*TI + p_ei_pgdp*P_GDP
TI ~ p_ti_isg*ISG
P_GDP ~ p_pgdp_to*TO
# indirect effect (a*b)
p_ei_ti_isg := p_ei_ti*p_ti_isg
p_ei_pgdp_to := p_ei_pgdp*p_pgdp_to
# total effect
p_ei_isg_total := p_ei_isg + (p_ei_ti*p_ti_isg)
p_ei_to_total := p_ei_to + (p_ei_pgdp*p_pgdp_to)
'

fit <- sem(model1, data=log_data, estimator='MLF', missing='ML')
summary(fit, standardized=TRUE, fit.measures=TRUE)

lavaan 0.6-5 ended normally after 64 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of free parameters                         12
                                                      
  Number of observations                            30
  Number of missing patterns                         2
                                                      
Model Test User Model:
                                                      
  Test statistic                                14.849
  Degrees of freedom                                 3
  P-value (Chi-square)                           0.002

Model Test Baseline Model:

  Test statistic                               226.218
  Degrees of freedom                                 9
  P-value                                        0.000

User Model versus Baseline Model:

  Comparative Fit Index (CFI)                    0.945
  Tucker-Lewis Index (TLI)            

In [122]:
standardizedSolution(fit, type='std.all')

lhs,op,rhs,est.std,se,z,pvalue,ci.lower,ci.upper
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EI,~,ISG,0.12182679,0.14379568,0.8472215,3.968717e-01,-0.160007565,0.40366115
EI,~,TO,0.07631719,0.16539021,0.4614371,6.444850e-01,-0.247841677,0.40047605
EI,~,TI,0.06639991,0.05277651,1.2581336,2.083434e-01,-0.037040160,0.16983997
EI,~,P_GDP,-1.21088741,0.10468955,-11.5664596,0.000000e+00,-1.416075146,-1.00569967
TI,~,ISG,0.80413800,0.10421405,7.7162147,1.199041e-14,0.599882214,1.00839379
P_GDP,~,TO,0.92482562,0.03853501,23.9996228,0.000000e+00,0.849298391,1.00035284
EI,~~,EI,0.01221976,0.00775111,1.5765170,1.149067e-01,-0.002972140,0.02741165
TI,~~,TI,0.35336207,0.16760496,2.1083032,3.500477e-02,0.024862386,0.68186175
P_GDP,~~,P_GDP,0.14469758,0.07127632,2.0300933,4.234705e-02,0.004998562,0.28439660


In [129]:
summary(lm(TI ~ ISG, data=log_data), standardized=TRUE)


Call:
lm(formula = TI ~ ISG, data = log_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.68860 -0.15671  0.02306  0.12760  0.48305 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -6.9097     1.7183  -4.021 0.000397 ***
ISG           3.8706     0.5407   7.158 8.64e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2792 on 28 degrees of freedom
Multiple R-squared:  0.6466,	Adjusted R-squared:  0.634 
F-statistic: 51.24 on 1 and 28 DF,  p-value: 8.642e-08
